# Inference + Intervals


In [ ]:
# Ensure local package is importable
import sys, subprocess

print('Python:', sys.executable)
subprocess.run(['pip', 'install', '-e', '.'], check=True)


Generate 24h forecasts with intervals from saved model bundles.

In [ ]:
import pandas as pd
from gridpulse.forecasting.predict import load_model_bundle, predict_next_24h

features = pd.read_parquet('data/processed/features.parquet')
bundle = load_model_bundle('artifacts/models/lstm_load_mw.pt')
forecast = predict_next_24h(features, bundle)
forecast


In [ ]:
import pandas as pd

# Build a frame for plotting
rows = pd.DataFrame({
    'timestamp': forecast['timestamp'],
    'p50': forecast['forecast'],
})
rows.head()
